# Fitzpatrick17k 데이터셋을 이용한 DermLIP 진단 평가

이 노트북은 Fitzpatrick17k 데이터셋에서 랜덤으로 1000개 이미지를 샘플링하여
DermLIP 모델로 진단하고, 진단 실패 케이스를 분석합니다.

In [ ]:
# 필수 패키지 설치
!pip install -q ftfy regex tqdm requests pandas pillow
!pip install -q git+https://github.com/openai/CLIP.git

In [ ]:
# Fitzpatrick17k 저장소 클론
!git clone https://github.com/mattgroh/fitzpatrick17k.git

In [ ]:
import pandas as pd
import numpy as np
import requests
from PIL import Image
from io import BytesIO
import clip
import torch
from tqdm.auto import tqdm
import os
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# DermLIP 모델 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"사용 디바이스: {device}")

# CLIP 모델 로드 (DermLIP은 CLIP 아키텍처 기반)
model, preprocess = clip.load("ViT-B/16", device=device)
print("DermLIP 모델 로드 완료!")

In [ ]:
# 46개 피부 질환 데이터베이스 (DermLIP 진단 시스템)
SKIN_DISEASE_DATABASE = {
    'acne': {
        'name_ko': '여드름',
        'name_en': 'Acne Vulgaris',
        'affected_areas': ['얼굴', '이마', '뺨', '턱', '등', '가슴'],
        'symptoms': ['붉은 반점', '염증', '농포', '검은 면포', '흰 면포', '피지 과다 분비'],
        'description': '모낭과 피지선의 만성 염증성 질환으로, 주로 사춘기에 발생하며 얼굴, 등, 가슴 부위에 면포, 구진, 농포, 결절 등이 나타납니다.',
        'severity_levels': ['경증 (면포성)', '중등도 (구진/농포성)', '중증 (결절성)', '최중증 (낭종성)']
    },
    'atopic_dermatitis': {
        'name_ko': '아토피 피부염',
        'name_en': 'Atopic Dermatitis',
        'affected_areas': ['팔꿈치 안쪽', '무릎 뒤쪽', '목', '손목', '발목', '얼굴'],
        'symptoms': ['가려움', '건조함', '붉은 반점', '진물', '피부 두꺼워짐', '피부 갈라짐'],
        'description': '만성 재발성 염증성 피부 질환으로, 심한 가려움증과 피부 건조증을 동반하며 주로 굽힘 부위에 습진성 병변이 나타납니다.',
        'severity_levels': ['경증', '중등도', '중증', '매우 중증']
    },
    'psoriasis': {
        'name_ko': '건선',
        'name_en': 'Psoriasis',
        'affected_areas': ['팔꿈치', '무릎', '두피', '허리', '손톱', '발톱'],
        'symptoms': ['은백색 각질', '붉은 반점', '가려움', '건조함', '출혈', '손발톱 변형'],
        'description': '만성 염증성 피부 질환으로, 은백색의 인설로 덮인 홍반성 구진 및 판이 전신에 나타나며 재발과 호전을 반복합니다.',
        'severity_levels': ['경증 (<3% BSA)', '중등도 (3-10% BSA)', '중증 (>10% BSA)', '농포성/홍피성']
    },
    'eczema': {
        'name_ko': '습진',
        'name_en': 'Eczema',
        'affected_areas': ['손', '발', '팔꿈치', '무릎', '목', '얼굴'],
        'symptoms': ['가려움', '붉은 반점', '물집', '진물', '딱지', '피부 건조'],
        'description': '피부의 표피에 발생하는 염증성 질환으로, 가려움증과 함께 홍반, 부종, 수포, 인설 등이 나타나는 피부염의 총칭입니다.',
        'severity_levels': ['급성기', '아급성기', '만성기']
    },
    'rosacea': {
        'name_ko': '주사',
        'name_en': 'Rosacea',
        'affected_areas': ['뺨', '코', '이마', '턱'],
        'symptoms': ['얼굴 붉어짐', '혈관 확장', '구진', '농포', '코 비대', '눈 자극'],
        'description': '주로 얼굴 중앙부에 발생하는 만성 염증성 피부 질환으로, 홍반, 모세혈관 확장, 구진, 농포 등이 나타납니다.',
        'severity_levels': ['홍반 혈관확장형', '구진 농포형', '비대형', '안구형']
    },
    'vitiligo': {
        'name_ko': '백반증',
        'name_en': 'Vitiligo',
        'affected_areas': ['손', '발', '얼굴', '입술', '겨드랑이', '사타구니'],
        'symptoms': ['흰색 반점', '색소 소실', '조기 백발', '점막 탈색'],
        'description': '멜라닌 세포의 소실로 인해 피부에 백색 반점이 나타나는 후천성 색소 탈실 질환입니다.',
        'severity_levels': ['국소형', '분절형', '전신형', '범발형']
    },
    'melasma': {
        'name_ko': '기미',
        'name_en': 'Melasma',
        'affected_areas': ['이마', '뺨', '윗입술', '코', '턱'],
        'symptoms': ['갈색 반점', '불규칙한 모양', '대칭적 분포', '경계 불분명'],
        'description': '주로 얼굴에 갈색의 색소 침착이 대칭적으로 나타나는 후천성 과색소침착 질환입니다.',
        'severity_levels': ['표피형', '진피형', '혼합형']
    },
    'seborrheic_dermatitis': {
        'name_ko': '지루성 피부염',
        'name_en': 'Seborrheic Dermatitis',
        'affected_areas': ['두피', '얼굴', '귀', '가슴', '등'],
        'symptoms': ['각질', '가려움', '붉은 반점', '기름진 인설', '비듬'],
        'description': '피지 분비가 많은 부위에 발생하는 만성 염증성 피부 질환으로, 홍반과 기름진 인설이 특징입니다.',
        'severity_levels': ['경증', '중등도', '중증']
    },
    'urticaria': {
        'name_ko': '두드러기',
        'name_en': 'Urticaria',
        'affected_areas': ['전신 어디든지'],
        'symptoms': ['팽진', '가려움', '붉은 반점', '부종', '따끔거림'],
        'description': '피부나 점막의 혈관 투과성 증가로 인해 일시적으로 팽진과 가려움증이 나타나는 질환입니다.',
        'severity_levels': ['급성 (<6주)', '만성 (≥6주)', '혈관부종 동반']
    },
    'contact_dermatitis': {
        'name_ko': '접촉성 피부염',
        'name_en': 'Contact Dermatitis',
        'affected_areas': ['접촉 부위'],
        'symptoms': ['가려움', '붉은 반점', '물집', '부종', '진물', '딱지'],
        'description': '외부 물질과의 접촉으로 인해 발생하는 염증성 피부 질환으로, 자극성과 알레르기성으로 구분됩니다.',
        'severity_levels': ['경증', '중등도', '중증', '만성화']
    },
    'fungal_infection': {
        'name_ko': '무좀',
        'name_en': 'Tinea Pedis',
        'affected_areas': ['발가락 사이', '발바닥', '손', '사타구니', '두피'],
        'symptoms': ['가려움', '각질', '물집', '짓무름', '악취', '발톱 변색'],
        'description': '피부사상균에 의한 표재성 진균 감염으로, 주로 발에 발생하며 가려움증과 인설이 특징입니다.',
        'severity_levels': ['지간형', '소수포형', '각화형', '염증형']
    },
    'warts': {
        'name_ko': '사마귀',
        'name_en': 'Verruca',
        'affected_areas': ['손', '발', '얼굴', '목'],
        'symptoms': ['융기된 병변', '거친 표면', '검은 점', '통증 (발바닥)'],
        'description': '인유두종 바이러스 감염에 의한 양성 피부 종양으로, 표면이 거칠고 융기된 병변이 나타납니다.',
        'severity_levels': ['단일형', '다발형', '융합형']
    },
    'herpes': {
        'name_ko': '헤르페스',
        'name_en': 'Herpes Simplex',
        'affected_areas': ['입술', '코 주변', '생식기', '엉덩이'],
        'symptoms': ['물집', '통증', '가려움', '따끔거림', '궤양', '발열'],
        'description': '단순 포진 바이러스 감염으로 인해 수포와 궤양이 발생하는 재발성 바이러스 감염 질환입니다.',
        'severity_levels': ['초발 감염', '재발성', '중증 (면역저하자)']
    },
    'shingles': {
        'name_ko': '대상포진',
        'name_en': 'Herpes Zoster',
        'affected_areas': ['몸통', '얼굴', '목', '팔', '다리'],
        'symptoms': ['통증', '물집', '발진', '신경통', '발열', '두통'],
        'description': '수두-대상포진 바이러스의 재활성화로 발생하는 질환으로, 신경 분포를 따라 통증과 수포가 나타납니다.',
        'severity_levels': ['경증', '중등도', '중증', '포진후 신경통']
    },
    'cellulitis': {
        'name_ko': '봉와직염',
        'name_en': 'Cellulitis',
        'affected_areas': ['다리', '발', '팔', '얼굴'],
        'symptoms': ['붉은 부종', '열감', '압통', '발열', '오한', '림프절 종대'],
        'description': '피부와 피하조직의 세균 감염으로, 국소적인 발적, 부종, 열감 및 전신 증상이 나타납니다.',
        'severity_levels': ['경증', '중등도', '중증', '괴사성']
    },
    'impetigo': {
        'name_ko': '농가진',
        'name_en': 'Impetigo',
        'affected_areas': ['얼굴', '코 주변', '입 주변', '손', '발'],
        'symptoms': ['물집', '농포', '꿀색 딱지', '가려움', '전염성'],
        'description': '주로 소아에서 발생하는 전염성이 강한 표재성 세균 감염으로, 특징적인 꿀색 딱지가 형성됩니다.',
        'severity_levels': ['국소형', '다발형', '수포형', '전염성 농가진']
    },
    'scabies': {
        'name_ko': '옴',
        'name_en': 'Scabies',
        'affected_areas': ['손가락 사이', '손목', '겨드랑이', '사타구니', '생식기'],
        'symptoms': ['심한 가려움 (밤)', '터널 모양 병변', '구진', '수포', '전염성'],
        'description': '옴진드기에 의한 전염성 피부 질환으로, 밤에 심해지는 가려움증과 특징적인 터널이 나타납니다.',
        'severity_levels': ['일반형', '결절형', '노르웨이형', '소아형']
    },
    'lice': {
        'name_ko': '이',
        'name_en': 'Pediculosis',
        'affected_areas': ['두피', '음모', '의류'],
        'symptoms': ['가려움', '서캐 (알)', '벌레 발견', '긁은 자국', '2차 감염'],
        'description': '이에 의한 기생충 감염으로, 가려움증과 함께 모발에 서캐가 붙어있는 것이 관찰됩니다.',
        'severity_levels': ['경증', '중등도', '중증 (2차 감염)']
    },
    'folliculitis': {
        'name_ko': '모낭염',
        'name_en': 'Folliculitis',
        'affected_areas': ['두피', '얼굴', '목', '가슴', '등', '엉덩이', '허벅지'],
        'symptoms': ['붉은 구진', '농포', '가려움', '압통', '모발 중심부'],
        'description': '모낭의 염증으로, 세균, 진균, 바이러스 감염이나 물리적 자극에 의해 발생합니다.',
        'severity_levels': ['표재성', '심재성', '만성 재발성']
    },
    'hidradenitis': {
        'name_ko': '화농성 한선염',
        'name_en': 'Hidradenitis Suppurativa',
        'affected_areas': ['겨드랑이', '사타구니', '엉덩이', '유방 밑'],
        'symptoms': ['통증성 결절', '농양', '누공', '흉터', '악취'],
        'description': '아포크린 땀샘 부위의 만성 재발성 염증 질환으로, 통증성 결절, 농양, 누공이 형성됩니다.',
        'severity_levels': ['Hurley I (경증)', 'Hurley II (중등도)', 'Hurley III (중증)']
    },
    'keratosis_pilaris': {
        'name_ko': '모공각화증',
        'name_en': 'Keratosis Pilaris',
        'affected_areas': ['팔 바깥쪽', '허벅지', '뺨', '엉덩이'],
        'symptoms': ['작은 돌기', '거친 피부', '건조함', '가려움 (경미)'],
        'description': '모낭 주위의 각질이 과다하게 축적되어 피부가 거칠어지는 양성 피부 질환입니다.',
        'severity_levels': ['경증', '중등도', '염증형']
    },
    'keratoacanthoma': {
        'name_ko': '각화극세포종',
        'name_en': 'Keratoacanthoma',
        'affected_areas': ['얼굴', '팔', '손'],
        'symptoms': ['빠른 성장', '중심부 각질', '화산 모양', '자연 퇴화 가능'],
        'description': '빠르게 성장하는 양성 피부 종양으로, 중심에 각질 마개가 있는 화산 모양의 결절이 특징입니다.',
        'severity_levels': ['단일형', '다발형', '거대형']
    },
    'actinic_keratosis': {
        'name_ko': '광선각화증',
        'name_en': 'Actinic Keratosis',
        'affected_areas': ['얼굴', '두피', '손등', '팔', '입술'],
        'symptoms': ['거친 각질', '붉은 반점', '딱지', '건조함', '압통'],
        'description': '장기간 자외선 노출로 인한 전암성 병변으로, 편평세포암으로 진행할 수 있습니다.',
        'severity_levels': ['Grade I (경증)', 'Grade II (중등도)', 'Grade III (중증)']
    },
    'seborrheic_keratosis': {
        'name_ko': '지루각화증',
        'name_en': 'Seborrheic Keratosis',
        'affected_areas': ['얼굴', '가슴', '등', '복부'],
        'symptoms': ['갈색/검은색 병변', '융기', '사마귀 같은 표면', '무통'],
        'description': '노화에 따라 발생하는 양성 표피 종양으로, 표면이 거칠고 색소가 침착된 병변이 특징입니다.',
        'severity_levels': ['평평형', '융기형', '각화형', '자극형']
    },
    'basal_cell_carcinoma': {
        'name_ko': '기저세포암',
        'name_en': 'Basal Cell Carcinoma',
        'affected_areas': ['얼굴', '코', '귀', '목', '두피'],
        'symptoms': ['진주 같은 구진', '궤양', '출혈', '딱지', '천천히 성장'],
        'description': '가장 흔한 피부암으로, 전이는 드물지만 국소적으로 파괴적이며 조기 치료가 중요합니다.',
        'severity_levels': ['표재형', '결절형', '침윤형', '경화형']
    },
    'squamous_cell_carcinoma': {
        'name_ko': '편평세포암',
        'name_en': 'Squamous Cell Carcinoma',
        'affected_areas': ['얼굴', '귀', '손', '팔', '입술'],
        'symptoms': ['거친 각질성 병변', '궤양', '출혈', '빠른 성장', '통증'],
        'description': '두 번째로 흔한 피부암으로, 전이 가능성이 있어 조기 발견과 치료가 필수적입니다.',
        'severity_levels': ['in situ', '침윤성', '고위험형', '전이성']
    },
    'melanoma': {
        'name_ko': '흑색종',
        'name_en': 'Melanoma',
        'affected_areas': ['등', '다리', '팔', '얼굴', '손발톱'],
        'symptoms': ['불규칙한 점', '색깔 변화', '크기 증가', '가려움', '출혈', 'ABCDE 징후'],
        'description': '멜라닌 세포에서 발생하는 악성 종양으로, 전이가 빠르고 치명적일 수 있어 조기 발견이 매우 중요합니다.',
        'severity_levels': ['Stage 0 (in situ)', 'Stage I-II (국소)', 'Stage III (림프절)', 'Stage IV (전이)']
    },
    'kaposi_sarcoma': {
        'name_ko': '카포시 육종',
        'name_en': 'Kaposi Sarcoma',
        'affected_areas': ['피부', '구강', '림프절', '내장'],
        'symptoms': ['자주색 반점', '결절', '부종', '출혈', '통증'],
        'description': '혈관 내피세포에서 발생하는 악성 종양으로, 주로 면역저하 환자에서 발생합니다.',
        'severity_levels': ['고전형', 'AIDS 관련형', '이식 관련형', '풍토병형']
    },
    'dermatofibroma': {
        'name_ko': '피부섬유종',
        'name_en': 'Dermatofibroma',
        'affected_areas': ['다리', '팔', '몸통'],
        'symptoms': ['단단한 결절', '갈색/붉은색', '움푹 들어감 (압박시)', '무통'],
        'description': '진피의 양성 섬유성 결절로, 단단하고 압박 시 중심부가 움푹 들어가는 특징이 있습니다.',
        'severity_levels': ['단일형', '다발형']
    },
    'lipoma': {
        'name_ko': '지방종',
        'name_en': 'Lipoma',
        'affected_areas': ['목', '어깨', '등', '팔', '허벅지'],
        'symptoms': ['부드러운 종괴', '이동 가능', '무통', '천천히 성장'],
        'description': '지방 조직의 양성 종양으로, 부드럽고 이동이 가능하며 통증은 없습니다.',
        'severity_levels': ['단일형', '다발형 (lipomatosis)']
    },
    'hemangioma': {
        'name_ko': '혈관종',
        'name_en': 'Hemangioma',
        'affected_areas': ['얼굴', '두피', '가슴', '등'],
        'symptoms': ['붉은/자주색 종괴', '부드러움', '크기 변화', '따뜻함'],
        'description': '혈관의 양성 증식으로, 주로 영유아기에 나타나며 대부분 자연 퇴화됩니다.',
        'severity_levels': ['표재형 (딸기)', '심재형', '혼합형', '합병증형']
    },
    'nevus': {
        'name_ko': '점 (모반)',
        'name_en': 'Melanocytic Nevus',
        'affected_areas': ['전신 어디든지'],
        'symptoms': ['갈색/검은색 반점', '균일한 색', '대칭', '경계 명확'],
        'description': '멜라닌 세포의 양성 증식으로, 선천적 또는 후천적으로 발생하는 색소성 병변입니다.',
        'severity_levels': ['접합형', '복합형', '진피내형', '이형성']
    },
    'skin_tag': {
        'name_ko': '연성섬유종 (쥐젖)',
        'name_en': 'Acrochordon',
        'affected_areas': ['목', '겨드랑이', '사타구니', '눈꺼풀'],
        'symptoms': ['작은 돌기', '살색/갈색', '부드러움', '줄기 있음'],
        'description': '피부에 매달린 작은 양성 종양으로, 마찰이 많은 부위에 주로 발생합니다.',
        'severity_levels': ['소형 (<2mm)', '중형 (2-5mm)', '대형 (>5mm)']
    },
    'cherry_angioma': {
        'name_ko': '체리 혈관종',
        'name_en': 'Cherry Angioma',
        'affected_areas': ['몸통', '팔', '다리'],
        'symptoms': ['밝은 붉은 색', '작은 크기', '돔 모양', '무통'],
        'description': '성인에서 흔히 발생하는 양성 혈관 증식으로, 선홍색의 작은 구진이 특징입니다.',
        'severity_levels': ['소수', '다발성']
    },
    'port_wine_stain': {
        'name_ko': '포도주색 모반',
        'name_en': 'Port Wine Stain',
        'affected_areas': ['얼굴', '목', '팔', '다리'],
        'symptoms': ['평평한 붉은/자주색 반점', '선천성', '점차 진해짐', '비대 가능'],
        'description': '선천적인 모세혈관 기형으로, 평평한 붉은색 또는 자주색 반점이 나이에 따라 진해집니다.',
        'severity_levels': ['연한 분홍색', '중간 붉은색', '진한 자주색', '결절 형성']
    },
    'cafe_au_lait': {
        'name_ko': '카페오레 반점',
        'name_en': 'Cafe au Lait Macule',
        'affected_areas': ['전신 어디든지'],
        'symptoms': ['연한 갈색 반점', '균일한 색', '타원형', '경계 명확'],
        'description': '우유에 커피를 탄 듯한 색의 색소성 반점으로, 다발성인 경우 신경섬유종증을 의심할 수 있습니다.',
        'severity_levels': ['단일형', '다발형 (신경섬유종증 의심)']
    },
    'mongolian_spot': {
        'name_ko': '몽고반점',
        'name_en': 'Mongolian Spot',
        'affected_areas': ['엉덩이', '허리', '등'],
        'symptoms': ['청회색 반점', '선천성', '점차 소실', '무통'],
        'description': '동양인에서 흔히 나타나는 선천적 색소성 병변으로, 대부분 성장하면서 자연 소실됩니다.',
        'severity_levels': ['전형적 위치', '비전형적 위치 (지속 가능)']
    },
    'xanthelasma': {
        'name_ko': '황색종',
        'name_en': 'Xanthelasma',
        'affected_areas': ['눈꺼풀'],
        'symptoms': ['노란색 판', '부드러움', '무통', '대칭적'],
        'description': '눈꺼풀에 발생하는 지질 침착으로, 고지혈증과 연관될 수 있습니다.',
        'severity_levels': ['국소형', '광범위형']
    },
    'keloid': {
        'name_ko': '켈로이드',
        'name_en': 'Keloid',
        'affected_areas': ['가슴', '어깨', '귀', '턱', '등'],
        'symptoms': ['융기된 흉터', '계속 자람', '가려움', '통증', '붉은색/어두운색'],
        'description': '상처 치유 과정에서 콜라겐이 과도하게 증식하여 원래 상처 범위를 넘어 자라는 병적 흉터입니다.',
        'severity_levels': ['소형', '중형', '대형', '거대형']
    },
    'hypertrophic_scar': {
        'name_ko': '비후성 반흔',
        'name_en': 'Hypertrophic Scar',
        'affected_areas': ['상처 부위'],
        'symptoms': ['융기된 흉터', '붉은색', '가려움', '시간이 지나면 개선'],
        'description': '상처 치유 과정에서 발생하는 융기된 흉터로, 켈로이드와 달리 원래 상처 범위를 넘지 않으며 시간이 지나면 개선됩니다.',
        'severity_levels': ['경증', '중등도', '중증']
    },
    'striae': {
        'name_ko': '튼살',
        'name_en': 'Striae Distensae',
        'affected_areas': ['복부', '가슴', '엉덩이', '허벅지', '팔'],
        'symptoms': ['선 모양 병변', '초기 붉은색', '나중 흰색', '약간 함몰'],
        'description': '피부가 급격히 늘어나면서 진피층이 파열되어 생기는 선상의 위축성 병변입니다.',
        'severity_levels': ['striae rubrae (붉은색)', 'striae albae (흰색)']
    },
    'burn': {
        'name_ko': '화상',
        'name_en': 'Burn',
        'affected_areas': ['노출 부위 또는 접촉 부위'],
        'symptoms': ['통증', '붉어짐', '물집', '피부 손상', '흉터'],
        'description': '열, 화학물질, 전기, 방사선 등에 의한 피부 손상으로, 깊이에 따라 1도, 2도, 3도로 분류됩니다.',
        'severity_levels': ['1도 (표피)', '2도 (진피)', '3도 (전층)', '4도 (근육/뼈)']
    },
    'pressure_ulcer': {
        'name_ko': '욕창',
        'name_en': 'Pressure Ulcer',
        'affected_areas': ['엉덩이', '꼬리뼈', '발뒤꿈치', '팔꿈치', '어깨'],
        'symptoms': ['피부 변색', '통증', '궤양', '괴사', '감염 위험'],
        'description': '장시간 압력으로 인한 피부와 조직의 손상으로, 주로 움직임이 제한된 환자에서 발생합니다.',
        'severity_levels': ['Stage 1 (홍반)', 'Stage 2 (부분층)', 'Stage 3 (전층)', 'Stage 4 (조직 손상)']
    },
    'leg_ulcer': {
        'name_ko': '하지궤양',
        'name_en': 'Leg Ulcer',
        'affected_areas': ['종아리', '발목', '발'],
        'symptoms': ['개방성 궤양', '통증', '부종', '변색', '느린 치유'],
        'description': '하지의 만성 궤양으로, 정맥, 동맥, 신경병성, 또는 혼합 원인에 의해 발생합니다.',
        'severity_levels': ['표재성', '심부형', '만성형', '감염형']
    },
    'diabetic_foot': {
        'name_ko': '당뇨발',
        'name_en': 'Diabetic Foot',
        'affected_areas': ['발', '발가락'],
        'symptoms': ['궤양', '감각 저하', '변색', '부종', '감염', '괴사'],
        'description': '당뇨병 합병증으로 발생하는 신경병증, 혈관병증, 감염이 복합된 발 질환입니다.',
        'severity_levels': ['Wagner 0-1 (경증)', 'Wagner 2-3 (중등도)', 'Wagner 4-5 (중증)']
    },
    'lupus': {
        'name_ko': '루푸스',
        'name_en': 'Cutaneous Lupus',
        'affected_areas': ['얼굴', '두피', '귀', '손', '가슴'],
        'symptoms': ['나비 모양 발진', '원판상 병변', '광과민성', '탈모', '구강 궤양'],
        'description': '자가면역 질환으로, 피부에 다양한 형태의 발진과 광과민성이 나타납니다.',
        'severity_levels': ['급성 피부 루푸스', '아급성 피부 루푸스', '만성 피부 루푸스']
    }
}

print(f"피부 질환 데이터베이스 로드 완료: {len(SKIN_DISEASE_DATABASE)}개 질환")

In [ ]:
# Fitzpatrick17k CSV 로드 및 샘플링
df = pd.read_csv('fitzpatrick17k/fitzpatrick17k.csv')
print(f"전체 데이터: {len(df)}개")
print(f"\n질병 종류: {df['label'].nunique()}개")
print(f"\n질병 분포 (상위 10개):")
print(df['label'].value_counts().head(10))

# 1000개 랜덤 샘플링
np.random.seed(42)
sampled_df = df.sample(n=1000, random_state=42).reset_index(drop=True)
print(f"\n샘플링된 데이터: {len(sampled_df)}개")

In [ ]:
# 질병 텍스트 프롬프트 생성 함수
def create_text_prompts():
    """46개 질병에 대한 텍스트 프롬프트 생성"""
    prompts = []
    disease_keys = []
    
    for key, info in SKIN_DISEASE_DATABASE.items():
        prompt = f"A photo of {info['name_en']}, a skin disease characterized by {', '.join(info['symptoms'][:3])}"
        prompts.append(prompt)
        disease_keys.append(key)
    
    return prompts, disease_keys

text_prompts, disease_keys = create_text_prompts()
print(f"생성된 프롬프트 수: {len(text_prompts)}개")
print(f"\n예시 프롬프트:")
for i in range(3):
    print(f"{i+1}. {text_prompts[i]}")

In [ ]:
# 텍스트 프롬프트 임베딩
print("텍스트 프롬프트 임베딩 중...")
text_tokens = clip.tokenize(text_prompts).to(device)
with torch.no_grad():
    text_features = model.encode_text(text_tokens)
    text_features /= text_features.norm(dim=-1, keepdim=True)
print("텍스트 임베딩 완료!")

In [ ]:
# 이미지 진단 함수
def diagnose_image(image_url):
    """URL에서 이미지를 다운로드하고 DermLIP으로 진단"""
    try:
        # 이미지 다운로드
        response = requests.get(image_url, timeout=10)
        if response.status_code != 200:
            return None, None, None
        
        image = Image.open(BytesIO(response.content)).convert('RGB')
        
        # 이미지 전처리 및 임베딩
        image_input = preprocess(image).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = model.encode_image(image_input)
            image_features /= image_features.norm(dim=-1, keepdim=True)
        
        # 유사도 계산
        similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
        values, indices = similarity[0].topk(3)
        
        # Top-1, Top-3 예측
        top1_disease = disease_keys[indices[0]]
        top1_confidence = values[0].item()
        
        top3_diseases = [(disease_keys[idx], values[i].item()) 
                        for i, idx in enumerate(indices)]
        
        return top1_disease, top1_confidence, top3_diseases
    
    except Exception as e:
        return None, None, None

In [ ]:
# Fitzpatrick17k label과 우리 시스템 매핑
LABEL_MAPPING = {
    'acne vulgaris': 'acne',
    'acne': 'acne',
    'atopic dermatitis': 'atopic_dermatitis',
    'psoriasis': 'psoriasis',
    'eczema': 'eczema',
    'rosacea': 'rosacea',
    'vitiligo': 'vitiligo',
    'melasma': 'melasma',
    'seborrheic dermatitis': 'seborrheic_dermatitis',
    'urticaria': 'urticaria',
    'allergic contact dermatitis': 'contact_dermatitis',
    'contact dermatitis': 'contact_dermatitis',
    'tinea pedis': 'fungal_infection',
    'tinea corporis': 'fungal_infection',
    'wart': 'warts',
    'verruca': 'warts',
    'herpes simplex': 'herpes',
    'herpes zoster': 'shingles',
    'cellulitis': 'cellulitis',
    'impetigo': 'impetigo',
    'scabies': 'scabies',
    'pediculosis': 'lice',
    'folliculitis': 'folliculitis',
    'hidradenitis': 'hidradenitis',
    'hidradenitis suppurativa': 'hidradenitis',
    'keratosis pilaris': 'keratosis_pilaris',
    'keratoacanthoma': 'keratoacanthoma',
    'actinic keratosis': 'actinic_keratosis',
    'actinic keratoses': 'actinic_keratosis',
    'seborrheic keratosis': 'seborrheic_keratosis',
    'basal cell carcinoma': 'basal_cell_carcinoma',
    'squamous cell carcinoma': 'squamous_cell_carcinoma',
    'melanoma': 'melanoma',
    'kaposi sarcoma': 'kaposi_sarcoma',
    'dermatofibroma': 'dermatofibroma',
    'lipoma': 'lipoma',
    'hemangioma': 'hemangioma',
    'nevus': 'nevus',
    'skin tag': 'skin_tag',
    'cherry angioma': 'cherry_angioma',
    'port wine stain': 'port_wine_stain',
    'cafe au lait macule': 'cafe_au_lait',
    'xanthelasma': 'xanthelasma',
    'keloid': 'keloid',
    'hypertrophic scar': 'hypertrophic_scar',
    'striae': 'striae',
    'burn': 'burn',
    'pressure ulcer': 'pressure_ulcer',
    'leg ulcer': 'leg_ulcer',
    'diabetic foot': 'diabetic_foot',
    'lupus': 'lupus',
    'cutaneous lupus': 'lupus'
}

def map_fitzpatrick_label(label):
    """Fitzpatrick17k 레이블을 우리 시스템에 매핑"""
    label_lower = label.lower().strip()
    return LABEL_MAPPING.get(label_lower, None)

In [ ]:
# 1000개 샘플 진단 수행
results = []

print("\n1000개 이미지 진단 시작...\n")
for idx, row in tqdm(sampled_df.iterrows(), total=len(sampled_df)):
    fitzpatrick_label = row['label']
    mapped_label = map_fitzpatrick_label(fitzpatrick_label)
    image_url = row['url']
    
    # DermLIP으로 진단
    top1_disease, top1_confidence, top3_diseases = diagnose_image(image_url)
    
    if top1_disease is None:
        continue
    
    # Top-1 정확도 체크
    top1_correct = (top1_disease == mapped_label) if mapped_label else False
    
    # Top-3 정확도 체크
    top3_correct = False
    if mapped_label:
        top3_predicted = [d[0] for d in top3_diseases]
        top3_correct = mapped_label in top3_predicted
    
    results.append({
        'index': idx,
        'url': image_url,
        'fitzpatrick_label': fitzpatrick_label,
        'mapped_label': mapped_label,
        'top1_predicted': top1_disease,
        'top1_confidence': top1_confidence,
        'top1_correct': top1_correct,
        'top3_predicted': str(top3_diseases),
        'top3_correct': top3_correct,
        'top1_disease_ko': SKIN_DISEASE_DATABASE[top1_disease]['name_ko'],
        'top1_disease_en': SKIN_DISEASE_DATABASE[top1_disease]['name_en']
    })

print(f"\n진단 완료: {len(results)}개 이미지")

In [ ]:
# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

# 1. 전체 진단 결과 CSV 저장
results_df.to_csv('diagnosis_results.csv', index=False, encoding='utf-8-sig')
print("\n✅ diagnosis_results.csv 저장 완료")
print(results_df.head())

In [ ]:
# 2. 진단 실패 케이스 분석
failed_df = results_df[results_df['top1_correct'] == False].copy()
print(f"\n진단 실패 케이스: {len(failed_df)}개")

# 실패 케이스 CSV 저장
failed_df.to_csv('failed_cases.csv', index=False, encoding='utf-8-sig')
print("\n✅ failed_cases.csv 저장 완료")
print(failed_df.head())

In [ ]:
# 3. 가장 많이 실패한 질병 Top 5 추출
# Fitzpatrick17k 원본 레이블 기준
failed_disease_counts = failed_df['fitzpatrick_label'].value_counts().head(5)

print("\n" + "="*60)
print("🔥 가장 많이 진단 실패한 질병 Top 5 (Fitzpatrick17k 기준)")
print("="*60)
for i, (disease, count) in enumerate(failed_disease_counts.items(), 1):
    print(f"{i}. {disease}: {count}건")

# Top 5 실패 질병 DataFrame
top5_failed = pd.DataFrame({
    'rank': range(1, 6),
    'disease': failed_disease_counts.index,
    'failure_count': failed_disease_counts.values
})

top5_failed.to_csv('top5_failed_diseases.csv', index=False, encoding='utf-8-sig')
print("\n✅ top5_failed_diseases.csv 저장 완료")

In [ ]:
# 4. 각 Top 5 실패 질병에 대한 상세 케이스 저장
for disease in failed_disease_counts.index:
    disease_failed = failed_df[failed_df['fitzpatrick_label'] == disease]
    filename = f"failures_{disease.replace(' ', '_').replace('/', '_')}.csv"
    disease_failed.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"✅ {filename} 저장 완료 ({len(disease_failed)}건)")

In [ ]:
# 5. 전체 통계 요약
total_evaluated = len(results_df)
top1_correct = results_df['top1_correct'].sum()
top3_correct = results_df['top3_correct'].sum()
top1_accuracy = (top1_correct / total_evaluated) * 100
top3_accuracy = (top3_correct / total_evaluated) * 100

summary_df = pd.DataFrame({
    'metric': ['Total Evaluated', 'Top-1 Correct', 'Top-1 Accuracy (%)', 
               'Top-3 Correct', 'Top-3 Accuracy (%)', 'Failed Cases'],
    'value': [total_evaluated, top1_correct, f"{top1_accuracy:.2f}",
              top3_correct, f"{top3_accuracy:.2f}", len(failed_df)]
})

summary_df.to_csv('summary.csv', index=False, encoding='utf-8-sig')
print("\n✅ summary.csv 저장 완료")

print("\n" + "="*60)
print("📊 평가 결과 요약")
print("="*60)
print(summary_df.to_string(index=False))
print("="*60)

In [ ]:
# 6. 시각화: 실패 질병 분포
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.barh(range(len(failed_disease_counts)), failed_disease_counts.values)
plt.yticks(range(len(failed_disease_counts)), failed_disease_counts.index)
plt.xlabel('Failure Count')
plt.title('Top 5 Most Failed Diseases in DermLIP Diagnosis')
plt.tight_layout()
plt.savefig('top5_failed_diseases.png', dpi=300, bbox_inches='tight')
print("\n✅ top5_failed_diseases.png 저장 완료")
plt.show()

## 결과 파일 설명

1. **diagnosis_results.csv**: 1000개 전체 진단 결과
2. **failed_cases.csv**: 진단 실패한 케이스만 필터링
3. **top5_failed_diseases.csv**: 가장 많이 실패한 질병 Top 5
4. **failures_[disease].csv**: 각 Top 5 질병별 상세 실패 케이스
5. **summary.csv**: 전체 평가 통계 요약
6. **top5_failed_diseases.png**: 실패 질병 분포 시각화